In [3]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt


# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('rsiperiod', 3),
    )
    # After trying with different time periods I found that for the given stocks, markets were susceptible
    # to correction after 1-3 days of extremely directed markets. Hence I used a rsi time period to be 3

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma= bt.indicators.RelativeStrengthIndex(
            self.datas[0], period=self.params.rsiperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        print()

    def next(self):

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            if  20 > self.sma[0]:

                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if 85 < self.sma[0]:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()                
                


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)
    
    datapath = ('TSLA.csv')

    # Create a Data Feed
    data = bt.feeds.GenericCSVData(
        dataname=datapath,
        dtformat=('%Y-%m-%d'),
        tmformat=('%H.%M.%S'),
        fromdate=datetime.datetime(2000, 1, 1),
       
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=100)

    # Set the commission
    cerebro.broker.setcommission(commission=0.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.plot()

Starting Portfolio Value: 100000.00
2019-09-24, BUY CREATE, 44.64
2019-09-25, BUY EXECUTED, Price: 44.91, Cost: 4491.20, Comm 0.00
2019-10-14, SELL CREATE, 51.39
2019-10-15, SELL EXECUTED, Price: 51.54, Cost: 4491.20, Comm 0.00
2019-10-15, OPERATION PROFIT, GROSS 662.80, NET 662.80

2019-11-22, BUY CREATE, 66.61
2019-11-25, BUY EXECUTED, Price: 68.86, Cost: 6886.40, Comm 0.00
2019-12-10, SELL CREATE, 69.77
2019-12-11, SELL EXECUTED, Price: 70.38, Cost: 6886.40, Comm 0.00
2019-12-11, OPERATION PROFIT, GROSS 151.20, NET 151.20

2020-02-26, BUY CREATE, 155.76
2020-02-27, BUY EXECUTED, Price: 146.00, Cost: 14600.00, Comm 0.00
2020-04-09, SELL CREATE, 114.60
2020-04-13, SELL EXECUTED, Price: 118.03, Cost: 14600.00, Comm 0.00
2020-04-13, OPERATION PROFIT, GROSS -2796.80, NET -2796.80

2020-08-10, BUY CREATE, 283.71
2020-08-11, BUY EXECUTED, Price: 279.20, Cost: 27920.00, Comm 0.00
2020-08-14, SELL CREATE, 330.14
2020-08-17, SELL EXECUTED, Price: 335.40, Cost: 27920.00, Comm 0.00
2020-08-17, 